<a href="https://colab.research.google.com/github/Psyclophe/Data-Scientist/blob/main/Wal_mart_analysis_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import datetime as dt
import prophet as ppt
import pickle as pkle
import warnings
#import pmdarima as pm
#from pmdarima.model_selection import train_test_split
#from pmdarima.pipeline import Pipeline
#from pmdarima.preprocessing import BoxCoxEndogTransformer
warnings.filterwarnings ("ignore")
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

ModuleNotFoundError: ignored

## sección 1.***Obtención de Datos***





In [ ]:
#se cargan los datos de entrada proporcionado en el proyecto de las tiendas Wal-mart, 
#asimismo se hace la conexión de los datos a Github mostrado en la siguiente ruta
#de la misma forma se indica a Python muestre los primeros "20" registros

url = 'https://raw.githubusercontent.com/Psyclophe/Data-Scientist/main/Walmart_Store_sales.csv'
df = pd.read_csv (url)

df.head(20)

## sección 2. **Análisis Exploratorio**
en esta sección del codigo se procede a realizar el análisis de la información proporcionada

In [ ]:
#para identificar los datos nulos se indica a Pthoin que nos muestre los datos y, poder detectar la cantidad de datos nulos

df.isnull().sum()

In [ ]:
#con la carga de información y con esta funcion se pide a Python indicar el tipo de dato de cada columna, 
#para analizar si es necesario hacer cambios en el tipo de dato en el dataframe y poder tener datos homógeneos
#se identifico que "Date" es un dato que es objeto y se debe convertir a dato fecha

df.info()

In [ ]:
#en esta función se pide a Python convertir los datos objeto a string
#aquí podemos visualizar que el dato que era un objeto "Date" se conviritio al formato fecha en donde se visualiza "datetime64[ns]"

df["Date"]=pd.to_datetime(df["Date"])

df.info()

In [ ]:
#con esta funcion se da la instrucción a Python para ver las medidas de tendencia central de las 8 columnas

#analizar holiday, temperatura del 75% al max

df.describe()

In [ ]:
#se da la instruccion a Python que se ordene las fechas de menor a mayor en el archivo que se va a analizar
#asimismo se indica al programa que ordene por fechas, de la mas antigua hasta la más reciente

data_sort=df.sort_values("Date")

data_sort

In [ ]:
df["Year"]= df['Date'].dt.year
df["Quarter"]= df['Date'].dt.Quarter
df["Month"]= df['Date'].dt.month
df["Week"]= df['Date'].dt.week
df["Day"]= df['Date'].dt.day
df

df['Year']=df['Date'].dt.year
df['Month']=df['Date'].dt.month
df['Day']=df['Date'].dt.day


#df["Week"]= pd.DatetimeIndex(df['Date']).week
#df['Month'] = pd.DatetimeIndex(df['Date']).month
#df['Year'] = pd.DatetimeIndex(df['Date']).year
#df

In [ ]:
#con la funcion "groupby" se indica a Python agrupar y sumar, por tienda y por semana, al ejecutar la función Python
#agrega e indexa la columna "0" a la columna "1" como identificador
#Las ventas estan expresadas en miles (´000)

group=df.groupby("Store").sum("Weekly_Sales")
group.reset_index(level=0, inplace=True)
group["Weekly_Sales"]=(group["Weekly_Sales"]/1000).round(2)
group["Fuel_Price"]=(group["Fuel_Price"]).round(2)
group["CPI"]=(group["CPI"]).round(2)
group["Unemployment"]=(group["Unemployment"]).round(2)

group

In [ ]:
#se solicita a Python ordenar por tienda y por volumen de ventas

sort_WeeklySales=group.sort_values('Weekly_Sales', ascending=False)

sort_WeeklySales

In [ ]:
#con este script le indicamos a Python que solo nos indique el Top_five de las ventas por tienda

Top_five=sort_WeeklySales[['Weekly_Sales', 'Store']][0:5:].reset_index(drop=True)
Top_five

In [ ]:
#es este script se solicita a Python calcular la Desviacioón Estandar por tienda, ordenanda de mayor a menor

Store_devstd = df.groupby('Store')['Weekly_Sales'].std()
Store_devstd = pd.DataFrame(Store_devstd)
Store_devstd=Store_devstd.sort_values('Weekly_Sales', ascending=False)

Store_devstd

In [ ]:
# con este script se determina el Coeficiente de Desviación Medio por tienda
#se determinó dividiendo la DesvStd por tienda entre la media de las tiendas

coeficiente_DM = pd.DataFrame(df.groupby('Store')['Weekly_Sales'].std() / df.groupby('Store')['Weekly_Sales'].mean()*100)
coeficiente_DM


In [ ]:
#con este script se pide a Python identificar los días festivos

df_Holiday=df[df["Holiday_Flag"]==1]

df_Holiday

In [ ]:
#con esta instrucción se definen los Días festivos que encontro Pyton en el dataframe

Super_Bowl =['2010-12-02', '2011-11-02', '2012-10-02']
Labor_Day =  ['2010-10-09', '2011-09-09', '2012-07-09']
Thanksgivig_Day =  ['2010-11-26', '2011-11-25', '2012-11-23']
Xmas = ['2010-12-31', '2011-12-30', '2012-12-28']

In [ ]:
#df.loc[df.Date.isin(Super_Bowl)].Date.unique()
#df.loc[df.Date.isin(Labor_Day)].Date.unique()
#df.loc[df.Date.isin(Thanksgivig_Day)].Date.unique()
#df.loc[df.Date.isin(Xmas)].Date.unique()

In [ ]:
#con esta instrucción se pide a Python determinar la media de ventas de los Días_Festivos

Sales_Super_Bowl= (pd.DataFrame(df.loc[df.Date.isin(Super_Bowl)]))['Weekly_Sales'].mean()
Sales_Labor_Day = (pd.DataFrame(df.loc[df.Date.isin(Labor_Day)]))['Weekly_Sales'].mean()
Sales_Thanksgivig_Day = (pd.DataFrame(df.loc[df.Date.isin(Thanksgivig_Day)]))['Weekly_Sales'].mean()
Sales_Xmas = (pd.DataFrame(df.loc[df.Date.isin(Xmas)]))['Weekly_Sales'].mean()

{Sales_Super_Bowl, Sales_Labor_Day, Sales_Thanksgivig_Day, Sales_Xmas}

In [ ]:
#con esta instrucción se pide a Python determinar la media de ventas de los Dias_No_Festivos

Sales_not_Holiday = df[df['Holiday_Flag'] == 0 ]['Weekly_Sales'].mean()

{Sales_not_Holiday}

In [ ]:
#con esta instrucción se pide a Python determinar el promedio de ventas 

Average_sales = {'Sales_Super_Bowl' : Sales_Super_Bowl,
                 'Sales_Labor_Day': Sales_Labor_Day,
                 'Sales_Thanksgiving_Day':Sales_Thanksgivig_Day,
                 'Sales_Xmas': Sales_Xmas,
                 'Sales_not_Holiday': Sales_not_Holiday}

Average_sales 

In [ ]:
#con este script se muestra en crecimiento tercer trimestre 2012 por tienda

third_quarter= df[(df['Date'] >= '2012-07-01') & (df['Date'] <= '2012-09-30')].groupby('Store')['Weekly_Sales'].sum().sort_values()
plt.figure(figsize=(15,7))
third_quarter.plot(kind='bar',legend=True)
plt.ticklabel_format( style='plain', axis='y')
plt.legend(["3rd-Quarter' 2012"]);


# sección 3. **Modelado**
en esta sección del codigo se procede a realizar el modelado y forecasting de la información proporcionada

In [ ]:
fig = px.box(data_sort, y = "Weekly_Sales", x = "Store",  color = "Store")
fig.show()

In [ ]:
sns.set_theme(style='darkgrid')
plt.figure(figsize=(30,12))
sns.lineplot(data=store, x='Date', y='Weekly_Sales', hue='Store', palette='tab10')
plt.show()

In [ ]:
f, ax=plt.subplots(figsize=(12,8))
sns.barplot(x='Store', y='Weekly_Sales', data=sort5, order=sort5.sort_values('Weekly_Sales').Store)
plt.show()

In [ ]:
df_Year=df.groupby('Year')['Weekly_Sales'].sum()
df_Year

In [ ]:
df[(df['Date'] - pd.to_datetime('2010-02-12')).abs().min()==(df['Date'] - pd.to_datetime('2010-02-12')).abs()]

In [ ]:
sns.set_theme(style='darkgrid')
plt.figure(figsize(20,8))
sns.lineplot(data=store1, x='Date', y='Weekly_Sales', hue='Store', palette='tab10')
plt.show()